In [1]:
%pip install -U pip
%pip install -U aioboto3 pandas requests pillow boto3

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
  Using cached pip-23.3.1-py3-none-any.whl.metadata (3.5 kB)
Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.
  Using cached aioboto3-12.0.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached pandas-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached Pillow-10.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.5 kB)
  Using cached boto3-1.28.81-py3-none-any.whl.metadata (6.7 kB)
  Using cached aiobotocore-2.7.0-py3-none-any.whl.metadata (20 kB)
  Using cached botocore-1.31.64-py3-

In [2]:
from utils import queue_jobs, list_all_objects, get_file_from_bucket
import pandas as pd
import json
import requests
from io import StringIO

asset_base_url = "https://salad-benchmark-assets.download/"
bucket_name = "salad-benchmark-public-assets"
clips_prefix = "cv-corpus-15.0-2023-09-08/en/clips/"
clip_duration_key = "cv-corpus-15.0-2023-09-08/en/clip_durations.tsv"
queue_url = "https://sqs.us-east-2.amazonaws.com/523358417554/benchmark-willow-whisper-large-v3.fifo"
clip_duration_file = asset_base_url + clip_duration_key
file = requests.get(clip_duration_file).text
clip_durations = pd.read_csv(StringIO(file), sep="\t")

clip_durations.head()

,clip,duration[ms]
0,common_voice_en_139641.mp3,4992
1,common_voice_en_28076481.mp3,4644
2,common_voice_en_544777.mp3,4560
3,common_voice_en_31080299.mp3,5040
4,common_voice_en_149757.mp3,2064


In [3]:
def get_jobs():
    for i, row in clip_durations.iterrows():
        id = row["clip"]
        key = clips_prefix + id
        download_url = asset_base_url + key
        duration = row["duration[ms]"]
        yield {
            "id": id,
            "download_url": download_url,
            "duration": duration
        }

# python is stupid slow at queueing jobs for some reason, so I dump this to a file and then use node to queue the jobs
with open("jobs.jsonl", "w") as f:
    for job in get_jobs():
        f.write(json.dumps(job) + "\n")